In [ ]:
import pandas as pd
import folium
import numpy as np
import math
from pathlib import Path

##############################
## Make changes here only: ###
##############################
out_path = Path('/Path/To/Out/')
data_path = Path('/Path/To/Metadata/')
meta_path = data_path.joinpath('Metadata_File.csv')
# Link IDs you wish to discard in advance
list_of_link_id_to_drop = ['4673-7HZ4','7HZ4-4673'] # examples
# Number of gridlines, equal for lats and lons
num_of_gridlines = 50
name_of_map_file = 'link_map1.html'
##############################
##############################

# No need to change anything from here onwards
df_md = pd.read_csv(meta_path)
df_md.drop_duplicates(subset='Link ID', inplace=True)
df_md.reset_index(inplace=True,drop=True)

grid = []
map_1 = folium.Map(location=[32, 35],
                       zoom_start=8,
                       tiles='Stamen Terrain',
                  control_scale=True)

for i,link in df_md.iterrows():
    if link['Link ID'] in list_of_link_id_to_drop:
        print('Link ID' + str(link['Link ID']) + ' has been dropped')
        continue
    if math.isnan(link['Rx Site Latitude']):
        print('No metadata for link ' + str(link['Link ID']))
        continue
    else:
        folium.PolyLine([(link['Rx Site Latitude'], 
                          link['Rx Site Longitude']),
                         (link['Tx Site Latitude'], 
                          link['Tx Site Longitude'])], 
                        color="purple", opacity=0.7, popup=str(link['Link ID'])).add_to(map_1)
        
# plot gridlines
lat_min = np.nanmin((np.nanmin(df_md['Tx Site Latitude'].values),
                    np.nanmin(df_md['Rx Site Latitude'].values)))
lon_min = np.nanmin((np.nanmin(df_md['Tx Site Longitude'].values),
                    np.nanmin(df_md['Rx Site Longitude'].values)))
lat_max = np.nanmax((np.nanmax(df_md['Tx Site Latitude'].values),
                    np.nanmax(df_md['Rx Site Latitude'].values)))
lon_max = np.nanmax((np.nanmax(df_md['Tx Site Longitude'].values),
                    np.nanmax(df_md['Rx Site Longitude'].values)))

lats = np.linspace(lat_min,lat_max,num_of_gridlines)
lons = np.linspace(lon_min,lon_max,num_of_gridlines)

for lat in lats:
    grid.append([[lat, -180],[lat, 180]])

for lon in lons:
    grid.append([[-90, lon],[90, lon]])

counter = 0
for g in grid:
    if counter < len (lats):
        folium.PolyLine(g, color="black", weight=0.5, opacity=0.5,popup=str(round(g[0][0],5))).add_to(map_1)
        counter += 1
    else: 
        folium.PolyLine(g, color="black", weight=0.5, opacity=0.5,popup=str(round(g[0][1],5))).add_to(map_1)
        
map_1.save(str(out_path.joinpath(name_of_map_file)))